In [400]:
#import necessary librabies for webscraping with selenium

import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import time
import requests
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Q1: Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [401]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#opening the page in automated chrome browser

driver.get("https://www.amazon.in/")

In [402]:
#Taking the input
user_input=input('Enter the Product which you want to search :')

Enter the Product which you want to search :Guitar


In [403]:
# finding element for job search bar
search=driver.find_element(By.XPATH,'//input[@id="twotabsearchtextbox"]')

# clearing any previous input in search bar

search.clear()

In [404]:
search.send_keys(user_input)

button=driver.find_element(By.XPATH,'//input[@id="nav-search-submit-button"]')
button.click()

# Q2: In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [417]:
# Make Empty list to scrap data
brand_name =[]
product = []
price =[]
exchange =[]
excepted_delivery =[]
availability =[]
url =[]

In [421]:
start= 0
end= 3
for page in range(start, end):
    url_tags =driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in url_tags:
        url.append(i.get_attribute('href'))
    time.sleep(2)
    
    # locating next page button and clicking
    
    next_page = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_page.click()
    time.sleep(6)

In [422]:
len(url)

189

In [423]:
for i in url:
    driver.get(i)
    time.sleep(3)
    
    # Extracting Brand Name
    try:
        brand_tags =driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[5]/div[4]/div[4]/div[43]/div/table/tbody/tr[1]/td[2]/span')
        brand_name.append(brand_tags.text) 
        
    except NoSuchElementException:
        brand_name.append('-')
    time.sleep(2)
    
    # Extracting product name
    try:
        product_tags =driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[5]/div[4]/div[4]/div[1]/div/h1/span')
        product.append(product_tags.text)
        
    except NoSuchElementException:
        product.append('-')
    time.sleep(2)
    
    # Extracting Price
    try:
        price_tags =driver.find_element(By.XPATH,'//span[@class="a-price a-text-price a-size-medium apexPriceToPay"]')
        price.append(price_tags.text)
        
    except NoSuchElementException:
        price.append('-')
    time.sleep(2)
    
    # Extracting Return or exchange details
    try:
        ex_tags =driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[5]/div[4]/div[4]/div[22]/div[2]/div/div/div/div[2]/span/div[2]/a')
        exchange.append(ex_tags.text)
        
    except NoSuchElementException:
        exchange.append('-')
    time.sleep(2)
    
    # Extracting Expected Delivery via Xpath
    try:
        delivery_tags =driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[5]/div[4]/div[1]/div[3]/div/div/div/div/form/div/div/div/div/div[3]/div/div[2]/div[8]/div[1]/div/div/div[2]/span/span[1]')
        excepted_delivery.append(delivery_tags.text)
        
    except NoSuchElementException:
        excepted_delivery.append('-')
    time.sleep(2)
    
    # Extracting availability via Xpath
    try:
        availability_tags =driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')
        availability.append(availability_tags.text)
        
    except NoSuchElementException:
        availability.append('-')
    time.sleep(2)

In [424]:
# Creating DataFrame for scrapped data

data_list= list(zip(brand_name,product,price, exchange,excepted_delivery,availability, url))

data= pd.DataFrame(data_list, 
    columns =['Brand_Name', 'Product', 'Price', 'Return/Exchange','Excepted Delivery','Availability','URL'],
    index=pd.RangeIndex(start=1, stop=190, name='Sr. No.'))

#Print the required data

print ("Details of each product listed in first 3 pages from amazon.com:\n")
data

Details of each product listed in first 3 pages from amazon.com:



,Brand_Name,Product,Price,Return/Exchange,Excepted Delivery,Availability,URL
Sr. No.,,,,,,,
1,-,Kadence Frontier guitar with Online Guitar lea...,-,-,-,,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,-,Kadence Slowhand Premium Jumbo Semi Acoustic G...,-,-,-,,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,-,Hontral S4 String Decor Guitar Children's Musi...,-,-,"Tuesday, 29 November",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,-,Vault DA40 41 Inch Premium Solid Spruce-Top Dr...,-,-,"Tuesday, 29 November",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
5,-,"Intern INT-38C Right hand Acoustic Guitar Kit,...",-,-,"Tuesday, 29 November",In stock.,https://www.amazon.in/Intern-INT-38C-Acoustic-...
...,...,...,...,...,...,...,...
185,-,-,-,-,-,In stock.,https://www.amazon.in/Epiphone-Classic-String-...
186,-,DEVICE OF URBAN INFOTECH Guitar Tuner Clip on ...,-,-,"Tuesday, 29 November",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
187,-,-,-,-,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [425]:
#Saving the data into a csv file
data.to_csv("Amazon_data.csv")

#Closing the driver
driver.close()

# Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [426]:
#Let's connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#open the page in automated chrome browser

driver.get("https://images.google.com/")
time.sleep(5)

# Scraping for fruits:

In [430]:
# Finding search

search=driver.find_element(By.XPATH,'//input[@class="gLFyf"]')

# clearing any previous input in search bar

search.clear()

# Puting 'fruits' to search button and click on seach

search.send_keys('fruits')
button=driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')
button.click()

In [431]:
# Scrolling window using ScrollBy

driver.execute_script("window.scrollBy(0,10000)","")

In [432]:
image_tag= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

url=[]
for img in image_tag:
    x= img.get_attribute('src')
    if x is not None:
        if(x[0:4] == 'http'):
            url.append(x)

In [433]:
len(url)

46

In [434]:
for i in range(len(url)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(url[i])
    file = open(r"F:\Webscraping\Fruits" + str(i)+".jpg","wb")
    file.write(response.content)

# Scraping for cars:

In [435]:
# Finding search

search=driver.find_element(By.XPATH,'/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input')

# clearing any previous input in search bar

search.clear()

# Puting 'cars' to search button and click on seach

search.send_keys('cars')
button=driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
button.click()

In [436]:
# Scrolling window using ScrollBy

driver.execute_script("window.scrollBy(0,10000)","")

In [437]:
image_tag= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

url=[]
for img in image_tag:
    x= img.get_attribute('src')
    if x is not None:
        if(x[0:4] == 'http'):
            url.append(x)

In [438]:
len(url)

38

In [439]:
for i in range(len(url)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(url[i])
    file = open(r"F:\Webscraping\Cars" +str(i)+".jpg","wb")
    file.write(response.content)

# Scraping for Machine Learning:-

In [440]:
# Finding search

search=driver.find_element(By.XPATH,'/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input')

# clearing any previous input in search bar

search.clear()

# Puting 'Machine Learning' to search button and click on seach

search.send_keys('Machine Learning')
button=driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
button.click()

In [441]:
# Scrolling window using ScrollBy

driver.execute_script("window.scrollBy(0,10000)","")

In [442]:
image_tag= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

url=[]
for img in image_tag:
    x= img.get_attribute('src')
    if x is not None:
        if(x[0:4] == 'http'):
            url.append(x)

In [443]:
len(url)

36

In [444]:
for i in range(len(url)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(url[i])
    file = open(r"F:\Webscraping\Machine Learning" +str(i)+".jpg","wb")
    file.write(response.content)

# Scraping for Guitar:-

In [447]:
# Finding search

search=driver.find_element(By.XPATH,"//input[@class='og3lId']")

# clearing any previous input in search bar

search.clear()

# Puting 'Guitar' to search button and click on seach

search.send_keys('Guitar')
button=driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
button.click()

In [448]:
# Scrolling window using ScrollBy

driver.execute_script("window.scrollBy(0,10000)","")

In [449]:
image_tag= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

url=[]
for img in image_tag:
    x= img.get_attribute('src')
    if x is not None:
        if(x[0:4] == 'http'):
            url.append(x)

In [450]:
len(url)

38

In [451]:
for i in range(len(url)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(url[i])
    file = open(r"F:\Webscraping\Guitar" +str(i)+".jpg","wb")
    file.write(response.content)

# Scraping for Cakes:-

In [452]:
# Finding search

search=driver.find_element(By.XPATH,'/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input')

# clearing any previous input in search bar

search.clear()

# Puting 'Cakes' to search button and click on seach

search.send_keys('Cakes')
button=driver.find_element(By.XPATH,'//div[@class="XZ5MVe"]')
button.click()

In [453]:
# Scrolling window using ScrollBy

driver.execute_script("window.scrollBy(0,10000)","")

In [454]:
image_tag= driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

url=[]
for img in image_tag:
    x= img.get_attribute('src')
    if x is not None:
        if(x[0:4] == 'http'):
            url.append(x)

In [455]:
len(url)

47

In [456]:
for i in range(len(url)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(url[i])
    file = open(r"F:\Webscraping\Cakes" +str(i)+".jpg","wb")
    file.write(response.content)

In [457]:
driver.close()

# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [458]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#open the page in automated chrome browser

driver.get("https://www.flipkart.com/")
time.sleep(2)

In [459]:
#close popup window

try:
    popup = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]')
    popup.click()
except NoSuchElementException:
    print('Popup Window is not present')

In [460]:
# Finding Search bar

search=driver.find_element(By.XPATH,'//input[@class="_3704LK"]')

# Clearing any previous input in search bar

search.clear()

# Putting 'Oneplus Nord' to search button and click on seach

search.send_keys('Oneplus Nord')
button=driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]')
button.click()

In [461]:
# Making empty list to scrape data
brand =[]
smartphone =[]
colour =[]
ram=[]
rom =[]
primary_camera=[]
dual_camera=[]
display_size=[]
battery_capacity=[]
price =[]
url=[]

In [462]:
url_tags =driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for i in url_tags:
    x=i.get_attribute('href')
    url.append(x)

In [463]:
len(url)

24

In [464]:
for i in url:
    driver.get(i)
    time.sleep(1)
    
    # Expanding specification table by clicking on read more button
    try:
        Read_more=driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')
        Read_more.click()
        
    except NoSuchElementException:
        print('NoSuchElementException Occur')
        pass
    
    # Extracting Brand Name
    try:
        brand_tags=driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')
        brand.append(brand_tags.text.split()[0])
    
    except NoSuchElementException:
        brand.append('-')
        
    # Extracting smartphone Name
    try:
        smartphone_tags=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')
        smartphone.append(smartphone_tags.text)
    
    except NoSuchElementException:
        smartphone.append('-')
        
    # Extracting Colour
    try:
        colour_tag =driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')
        colour.append(colour_tag.text)
    
    except NoSuchElementException:
        colour.append('-')
        
    # Extracting Storage ram
    try:
        ram_tag =driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table/tbody/tr[2]/td[2]/ul/li')
        ram.append(ram_tag.text)
        
    except NoSuchElementException:
        ram.append('-')
    
        
    # Extracting Storage rom
    try:
        rom_tag =driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table/tbody/tr[1]/td[2]/ul/li')
        rom.append(rom_tag.text)
        
    except NoSuchElementException:
        rom.append('-')
        
    # Extracting Primary camera details
    p=[]
    try:
        camera_tags=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table/tbody/tr[1]/td[2]/ul/li')
        p.append(camera_tags.text)
        for i in p:
            if i=='Yes':
                primary_camera.append('--')
            else:
                primary_camera.append(i)
                
    except NoSuchElementException:
        primary_camera.append('-')
                
        
    # Extracting Dual camera lens details
    try:
        dual_camera_tags=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table/tbody/tr[2]/td[2]/ul/li')
        dual_camera.append(dual_camera_tags.text)
    
    except NoSuchElementException:
        dual_camera.append('-')
     
     
    # Extracting Display size details
    try:
        display_size_tags=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table/tbody/tr[1]/td[2]/ul/li')
        display_size.append(display_size_tags.text)
    
    except NoSuchElementException:
        display_size.append('-')
      
      
    # Extracting battery capacity details
    l=[]
    try:
        battery_capacity_tags=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][7]/table/tbody/tr/td[2]/ul/li')
        l.append(battery_capacity_tags.text)
        for i in l:
            if len(i)== 8:
                battery_capacity.append(i)
            else:
                battery_capacity.append("--")
    
    except NoSuchElementException:
        battery_capacity.append('-')
        
        
    # Extracting price details
    try:
        price_tags=driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
        price.append(price_tags.text)
    
    except NoSuchElementException:
        price.append('-')

In [465]:
# Creating DataFrame for scrapped data

data_list= list(zip(brand,smartphone,colour, ram,rom,primary_camera,dual_camera,display_size,battery_capacity,price,url))

data= pd.DataFrame (data_list,
    columns =['Brand_Name', 'Smartphone Name', 'Colour', 'RAM','Storage(ROM)', 'Primary Camera','Dual Camera',
              'Display Size', 'Battery Capacity','Price', 'URL'],
    index = pd.RangeIndex(start=1, stop=25, name='Sr. No.'))

#Print the required data

data.to_csv("oneplus_details.csv", index= False)

print("Details of each product listed in first page from flipkart.com:\n")
data

Details of each product listed in first page from flipkart.com:



,Brand_Name,Smartphone Name,Colour,RAM,Storage(ROM),Primary Camera,Dual Camera,Display Size,Battery Capacity,Price,URL
Sr. No.,,,,,,,,,,,
1,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,64MP Rear Camera,Primary & Secondary Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,970",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,--,64MP Rear Camera,16.74 cm (6.59 inch),--,"₹19,799",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,64MP Rear Camera,Primary & Secondary Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,692",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,64MP Rear Camera,Primary & Secondary Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,730",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,64MP Rear Camera,Primary & Secondary Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,892",https://www.flipkart.com/oneplus-nord-ce-2-lit...
6,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,64MP Rear Camera,Primary & Secondary Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,948",https://www.flipkart.com/oneplus-nord-ce-2-lit...
7,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,64MP Rear Camera,Primary Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,899",https://www.flipkart.com/oneplus-nord-ce-2-lit...
8,OnePlus,Nord N20 SE,Blue Oasis,4 GB,64 GB,50MP Rear Camera,Primary Camera,16.66 cm (6.56 inch),--,"₹14,990",https://www.flipkart.com/oneplus-nord-n20-se-b...
9,OnePlus,Nord N20 SE,Celestial Black,4 GB,64 GB,50MP Rear Camera,Primary Camera,16.66 cm (6.56 inch),5000 mAh,"₹14,748",https://www.flipkart.com/oneplus-nord-n20-se-c...


In [466]:
driver.close()

# Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [467]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#open the page in automated chrome browser

driver.get('https://www.google.co.in/maps')
time.sleep(2)

In [468]:
# Finding Search bar

search=driver.find_element(By.XPATH,'//input[@jslog="11886"]')

# Clearing any previous input in search bar

search.clear()

# Putting 'Varanasi' to search button and click on seach

search.send_keys('Varanasi')
button=driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button')
button.click()

In [469]:
# finding the current url

current_URL=driver.current_url

print('Current url for given location is :',current_URL)

Current url for given location is : https://www.google.co.in/maps/place/Varanasi,+Uttar+Pradesh/@25.3102794,82.9955222,14z/data=!4m5!3m4!1s0x398e2db76febcf4d:0x68131710853ff0b5!8m2!3d25.3176452!4d82.9739144


In [470]:
# finding total location from URL

if "@" in current_URL:
        loc = current_URL.split('@')[1].split(',*/data')[0].split(',')
loc        

['25.3102794',
 '82.9955222',
 '14z/data=!4m5!3m4!1s0x398e2db76febcf4d:0x68131710853ff0b5!8m2!3d25.3176452!4d82.9739144']

In [471]:
# scraping latitude and longitude from URL

try:
    if "@" in current_URL:
        loc = current_URL.split('@')[1].split(',*/data')[0].split(',')
        print('Latitude of given Location:',loc[0])
        print('Longitude of given Location:',loc[1])
except:
    print('Location details is not found in URL')

Latitude of given Location: 25.3102794
Longitude of given Location: 82.9955222


In [472]:
driver.close()

# Q6.Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [473]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#open the page in automated chrome browser

driver.get('https://www.trak.in')
time.sleep(2)

In [474]:
# Clicking 'funding deals'

funding_deals_tags = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a')
x= funding_deals_tags.get_attribute('href')
driver.get(x)

In [475]:
# Making Empty list
date=[]
startup=[]
industry=[]
sub_vertical=[]
city=[]
investor=[]
investment_type=[]
amount=[]

In [476]:
for i in range(54,57):
    
    # Extracting Date
    try:
        date_tags =driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
        for i in date_tags:
            date.append(i.text)
    except:
        date.append('--')

In [477]:
for i in range(54,57):
        
    # Extracting startup name
    try:
        startup_tags =driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
        for i in startup_tags:
            startup.append(i.text)
    except:
        startup.append('--')

In [478]:
for i in range(54,57):

    # Extracting Industry
    try:
        industry_tags= driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
        for i in industry_tags:
            industry.append(i.text)
    except:
        industry.append('--')

In [479]:
for i in range(54,57):

    # Extracting subvertical
    try:
        subvertical_tags =driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
        for i in subvertical_tags:
            sub_vertical.append(i.text)
    except:
        sub_vertical.append('--')

In [480]:
for i in range(54,57):

    # Extracting city/location
    try:
        city_tags =driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
        for i in city_tags:
            city.append(i.text)
    except:
        city.append('--')

In [481]:
for i in range(54,57):

    # Extracting Investor
    try:
        investor_tags =driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
        for i in investor_tags:
            investor.append(i.text)
    except:
        investor.append('--')

In [482]:
for i in range(54,57):   
    
    # Extracting Investment type
    try:
        investment_tags=driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
        for i in investment_tags:
            investment_type.append(i.text)
    except:
        investment_type.append('--')

In [483]:
for i in range(54,57): 
    
    # Extracting Amount
    try:
        amount_tags =driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
        for i in amount_tags:
            amount.append(i.text)
    except:
        amount.append('--')

In [484]:
# Creating DataFrame for scrapped data

data_list= list(zip(date,startup,industry,sub_vertical,city,investor,investment_type,amount))

data= pd.DataFrame (data_list,
    columns =['Date(dd/mm/yyyy)', 'Startup Name', 'Industry/Vertical', 'Sub-Vertical','City/Location','Investor Name',
              'Investment Type', 'Amount(In USD)'])

#Print the required data

data.to_csv("Trak_in.csv", index= True)

print("Details of all the funding deals for second quarter (i.e. Jan 21 – March 21) from trak.in:\n")
data

Details of all the funding deals for second quarter (i.e. Jan 21 – March 21) from trak.in:



,Date(dd/mm/yyyy),Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor Name,Investment Type,Amount(In USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [485]:
driver.close()

# Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [486]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#open the page in automated chrome browser

driver.get('https://www.digit.in')
time.sleep(2)

In [487]:
# Clicking on top 10 option button in menu bar

top10= driver.find_element(By.XPATH,'//div[@class="menu"]/ul/li[4]/a')
top10.click()
time.sleep(2)

# Clicking on Laptop
lap= driver.find_element(By.XPATH,'//div[@class="categoty_list"]/button[2]')
lap.click()

# Clicking on Best gaming laptops

gaming_lap =driver.find_element(By.XPATH,'//div[@id="laptops"]/div[3]/a').get_attribute('href')
driver.get(gaming_lap)
time.sleep(3)

In [488]:
# Making Empty list for scraping data

laptop_model =[]
processor =[]
display = []
OS =[]
memory =[]
graphics_processor =[]
dimension =[]
weight =[]
url=[]

In [489]:
# Extracting Laptop model

laptop_tags =driver.find_elements(By.XPATH, '//div[@class="left_side"]/a/h3')
for i in laptop_tags:
    laptop_model.append(i.text)

# Extracting processor

processor_tags =driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
for i in processor_tags:
    processor.append(i.text)
    
# Extracting display

display_tags =driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
for i in display_tags:
    display.append(i.text)

# Extracting OS

OS_tags =driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
for i in OS_tags:
    OS.append(i.text)    

# Extracting memory

memory_tags =driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')
for i in memory_tags:
    memory.append(i.text)
    
# Extracting graphics_processor

graphics_processor_tags =driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
for i in graphics_processor_tags:
    graphics_processor.append(i.text)
    
# Extracting dimension

dimension_tags =driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
for i in dimension_tags:
    dimension.append(i.text.split("&")[0])
    
# Extracting weight

weight_tags =driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
for i in weight_tags:
    weight.append(i.text.split("&")[1])
    
# Extracting url details

url_tags =driver.find_elements(By.XPATH,'//div[@class="left_side"]/a')
for i in url_tags:
    url.append(i.get_attribute("href"))

In [490]:
# Creating DataFrame for scrapped data

data_list= list(zip(laptop_model,processor,display,OS,memory,graphics_processor,dimension,weight,url))

data= pd.DataFrame (data_list,
    columns =['Laptop Model', 'Processor', 'Display', 'OS','Memory','Graphics Processor',
              'Dimension', 'Weight', 'URL'], index= pd.RangeIndex(start=1, stop=11, name='Sr. No.'))

#Print the required data

data.to_csv("Digit_laptop_details.csv", index= False)

print("Details of all the available details of best gaming laptops from digit.in:\n")
data

Details of all the available details of best gaming laptops from digit.in:



,Laptop Model,Processor,Display,OS,Memory,Graphics Processor,Dimension,Weight,URL
Sr. No.,,,,,,,,,
1,MSI Titan GT77-12UHS,12th Gen Intel Core i9-12900HX 16 core processor,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension,3.3 kg weight,https://www.digit.in/laptops/msi-titan-gt77-12...
2,Alienware X17 R2,12th Gen Intel Core i9-12900H 14 core processo...,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,299 x 359 x 21 mm dimension,2.96 kg weight,https://www.digit.in/laptops/dell-new-alienwar...
3,Acer Predator Triton 500 SE PT516-52s,12th Gen Intel Core i7-12700H 14 core processo...,16″ (2560 x 1600) screen,Windows 11,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,262 x 359 x 20 mm dimension,2.4 kg weight,https://www.digit.in/laptops/acer-predator-tri...
4,Omen By HP (16-B1371TX),12th Gen Intel Core i7-12700H 14 core processo...,16.1″ (2560 x 1440) screen,Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension,2.32 kg weight,https://www.digit.in/laptops/omen-by-hp-16-b13...
5,Acer Predator Helios 300 AN515-45,AMD Ryzen 9-5900HX 8 core processor,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension,2.4 kg weight,https://www.digit.in/laptops/acer-nitro-5-an51...
6,MSI Delta 15 (A5EFK-083IN),AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension,1.9 kg weight,https://www.digit.in/laptops/msi-delta-15-5th-...
7,Omen By HP (16-C0141AX),AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16.1″ (2560 x 1440) screen,Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension,2.3 kg weight,https://www.digit.in/laptops/hp-omen-16-c0141a...
8,Lenovo Legion 5i Pro (82RF00MGIN),12th Gen Intel Core i7-12700H 14 core processo...,"16″ (2560 x 1600) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,360 x 264 x 20 mm dimension,2.49 kg weight,https://www.digit.in/laptops/lenovo-legion-5-p...
9,Alienware M15 R5 Ryzen Edition Icc-C780001win,AMD Ryzen R7-5800H 8 core processor with 4.40 ...,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension,2.69 kg weight,https://www.digit.in/laptops/dell-alienware-m1...


In [491]:
driver.close()

# Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [492]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#get the specified url
url = "https://www.forbes.com/?sh=41bd46d2254c"
driver.get(url)
time.sleep(3)

In [500]:
#let's get option button from the page
opt_btn = driver.find_element(By.XPATH,"//div[@class='header__left']")
opt_btn.click()
time.sleep(3)

#select billionaires from options
blns = driver.find_element(By.XPATH,"//div[@role='button']")
blns.click()
time.sleep(3)
#select world billionaire
bln_list = driver.find_element(By.XPATH,"//li[@class='TjJgrPSg _2bNo56RE secondary']")
bln_list.click()
time.sleep(4)

In [501]:
# Empty list to scrap data
Rank = []
Person_Name = []
Net_worth = []
Age = []
Citizenship = []
Source = []
Industry = []

In [503]:
while(True):
    
    # scraping the data of rank of the billionaires
    rank_tag = driver.find_elements(By.XPATH,"//div[@class='rank']")
    for rank in rank_tag:
        Rank.append(rank.text)
    time.sleep(1)
    
    
 
    # scraping the data  of names of the billionaires
    name_tag = driver.find_elements(By.XPATH,"//div[@class='personName']/div")
    for name in name_tag:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    # scraping the data of age of the billionaires
    age_tag = driver.find_elements(By.XPATH,"//div[@class='age']/div")
    for age in age_tag:
        Age.append(age.text)
    time.sleep(1)
    
    
    # scraping the data of citizenship of the billionaires
    cit_tag = driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
    for cit in cit_tag:
        Citizenship.append(cit.text)
    time.sleep(1)
    
    
    # scraping the data of source of income of the billionaires
    sour_tag = driver.find_elements(By.XPATH,"//div[@class='source']")
    for sour in sour_tag:
        Source.append(sour.text)
    time.sleep(1)
    
    
    # scraping data of industry of the billionaires
    ind_tag = driver.find_elements(By.XPATH,"//div[@class='category']//div")
    for ind in ind_tag:
        Industry.append(ind.text)
    time.sleep(1)
    
    
    # scraping data of net_worth of billionaires
    net_tag = driver.find_elements(By.XPATH,"//div[@class='netWorth']/div")
    for net in net_tag:
        Net_worth.append(net.text)
    time.sleep(1)
    
    
    # clicking on next button
    try:
        next_button = driver.find_element(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break

In [504]:
# printing length
print(len(Rank),len(Person_Name),len(Net_worth),len(Age),len(Citizenship),len(Source),len(Industry))

2668 2668 2668 2668 2668 2668 2668


In [505]:
# Creating DataFrame for scrapped data

Billionaires = pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Name'] = Person_Name
Billionaires['Net Worth'] = Net_worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = Citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = Industry
Billionaires

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,,United States,,
1,2.,Jeff Bezos,$171 B,,United States,,
2,3.,Bernard Arnault & family,$158 B,,France,,
3,4.,Bill Gates,$129 B,,United States,,
4,5.,Warren Buffett,$118 B,,United States,,
...,...,...,...,...,...,...,...
2663,2578.,Zhang Yuqiang,$1 B,66,China,Fiberglass,Manufacturing
2664,2578.,Zhou Ruxin,$1 B,59,China,Navigation,Technology
2665,2578.,Wen Zhou & family,$1 B,57,China,chemicals,Manufacturing
2666,2578.,Zhou Yifeng & family,$1 B,43,China,liquefied petroleum gas,Energy


In [506]:
# saving dataset in csv
Billionaires.to_csv('Forbes_Billionaires.csv')

In [507]:
driver.close()

# Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [529]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#open the page in automated chrome browser

driver.get('https://www.youtube.com')
time.sleep(2)

In [530]:
# Finding search menu
search=driver.find_element(By.XPATH,'//input[@id="search"]') 

#Putting input video name by user

search.send_keys('Babbu Maan : Adab Punjabi | Pagal Shayar')

# Finding Search button and clicking

button=driver.find_element(By.XPATH,'//html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/button')  
button.click()
time.sleep(2)

In [532]:
# Clicking on video

search_video = driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div[2]/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/div/div[1]/div/h3/a/yt-formatted-string')
search_video.click()
time.sleep(2)

In [533]:
# creating empty lists to save the data
comment = []
upvote = []
timeposted = []

In [534]:
from time import sleep

# Loading all comments
i = 1 
while True:
    i += 500
    driver.execute_script(f"window.scrollTo(0, {i});")
        
    all_box = driver.find_elements(By.XPATH,'//ytd-comment-thread-renderer')
    
    sleep(1)
    
    if len(all_box) > 500:
        break

In [535]:
full = []
for i in all_box:
    comment = i.find_element(By.XPATH,'.//ytd-expander[@id="expander"]').text
    upvote = i.find_element(By.XPATH,'.//span[@id="vote-count-middle"]').text
    time = i.find_element(By.XPATH,'.//div[@id="header-author"]//yt-formatted-string/a').text
    
    full.append({
        'comment' : comment,
        'upvote' : upvote,
        'time' : time
    })
    
data = pd.DataFrame(full)
data

,comment,upvote,time
0,ਪੰਜਾਬੀ ਇੰਡਸਟਰੀ ਦਾ ਸਭ ਤੋਂ ਵੱਡਾ ਗਾਣਾ ਕੋਂਣ ਕੋਂਣ ਮ...,1K,1 year ago
1,"7ਮਹੀਨੇ ਬਾਅਦ ਵੀ ਸੋਂਗ ਸੁਣ ਕੇ ਨਵਾ ਨਵਾ ਜਿਹਾ ਲਗਦਾ ,...",39,11 months ago
2,ਪੰਜਾਬੀ ਇੰਡਸਟਰੀ ਦਾ ਮਹਾਨ ਸਿੰਗਰ ਸਿਰਫ ਨਾਮ ਹੀ ਕਾਫੀ ...,110,1 year ago
3,ਮੇਰੀ ਜਾਨ BABBU MAAN,3,4 months ago
4,10 million cross \nਅਸਲੀ ਹੈ ਬਾਈ ਅਸਲੀ \nਮਾਨ ਸਾਹਬ,169,1 year ago
...,...,...,...
515,I m big fan man Shab from himachal,3,4 months ago
516,ਵੀਰੋ ਅਸਲੀ 6 ਮਿਲੀਅਨ ਲਈ ਸਾਰੇ ਕੱਟੜ ਫੈਨ ਤੇ ਮਾਨ ਸਾਬ...,84,1 year ago
517,Geet sunke kayia de mirch lagdi a,,8 months ago
518,ਬੱਬੂ ਮਾਨ ਬਹੁਤ ਵਧੀਆ ਗਾਣਾ,,1 year ago


In [536]:
data_final= data[0:500]
#Print the required data

data_final.to_csv("YTSong_details.csv", index= True)

print("Details of  500 Comments, upvote and time when comment was posted from any YouTube Video:\n")
data_final

Details of  500 Comments, upvote and time when comment was posted from any YouTube Video:



,comment,upvote,time
0,ਪੰਜਾਬੀ ਇੰਡਸਟਰੀ ਦਾ ਸਭ ਤੋਂ ਵੱਡਾ ਗਾਣਾ ਕੋਂਣ ਕੋਂਣ ਮ...,1K,1 year ago
1,"7ਮਹੀਨੇ ਬਾਅਦ ਵੀ ਸੋਂਗ ਸੁਣ ਕੇ ਨਵਾ ਨਵਾ ਜਿਹਾ ਲਗਦਾ ,...",39,11 months ago
2,ਪੰਜਾਬੀ ਇੰਡਸਟਰੀ ਦਾ ਮਹਾਨ ਸਿੰਗਰ ਸਿਰਫ ਨਾਮ ਹੀ ਕਾਫੀ ...,110,1 year ago
3,ਮੇਰੀ ਜਾਨ BABBU MAAN,3,4 months ago
4,10 million cross \nਅਸਲੀ ਹੈ ਬਾਈ ਅਸਲੀ \nਮਾਨ ਸਾਹਬ,169,1 year ago
...,...,...,...
495,Ustaad️ babbu,,1 month ago
496,Love you babbuman ustad legend ️️,,7 months ago
497,Bhot sona ️️,,4 months ago
498,ਬੱਬੂ ਮਾਨ ਹਮੇਸ਼ਾ ਟਾਪ ਤੇ ਹੀ ਰਹੂਗਾ ️️️,171,1 year ago


In [537]:
driver.close()

# Q10.Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [508]:
#First connect to the driver

driver= webdriver.Chrome(r'C:\Users\Arpita\Downloads\chromedriver_win32\chromedriver.exe')

#open the page in automated chrome browser

driver.get('https://www.hostelworld.com/')
time.sleep(2)

In [509]:
# Finding search menu

search = driver.find_element(By.XPATH,'//input[@id="location-text-input-field"]')
search.send_keys("London")

In [511]:
# clicking london

london = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
london.click()

In [514]:
#search for london

letsgo= driver.find_element(By.XPATH,'//div[@class="search-button"]')
letsgo.click()

In [515]:
#empty list for storing required data

hostel_name = []
distance = []
facilities =[]
url = []

In [516]:
#fetching hostel name
try:
    name_tags = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]/a')
    for i in name_tags:
        hostel_name.append(i.text)
except NoSuchElementException:
    hostel_name.append('-')

In [517]:
len(hostel_name)

19

In [518]:
#fetching distance
try:
    distance_tags = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for i in distance_tags:
        distance.append(i.text.split("-")[1])
except NoSuchElementException:
    distance.append('-')

In [519]:
len(distance)

19

In [520]:
url_tags =driver.find_elements(By.XPATH,'//h2[@class="title title-6"]/a')
for i in url_tags:
    x=i.get_attribute('href')
    url.append(x)
url

['https://www.hostelworld.com/pwa/hosteldetails.php/Hostel-One-Notting-Hill/London/291433?from=2022-11-25&to=2022-11-28&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Village/London/502?from=2022-11-25&to=2022-11-28&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Generator-London/London/510?from=2022-11-25&to=2022-11-28&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Safestay-London-Kensington-Holland-Park/London/93920?from=2022-11-25&to=2022-11-28&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Camden/London/505?from=2022-11-25&to=2022-11-28&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/NX-London-Hostel/London/15204?from=2022-11-25&to=2022-11-28&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Hammersmith/London/20976?from=2022-11-25&to=2022-11-28&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Smart-Hyde-Park-View-Hostel/London/

In [521]:
len(url)

19

In [522]:
#fetching distance
try:
    facilities_tags = driver.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for i in facilities_tags:
        facilities.append(i.text.replace("\n",","))
except NoSuchElementException:
    facilities.append('-')
facilities

['Free WiFi',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Free Breakfast',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Free Breakfast,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi,Follows Covid-19 sanitation guidance',
 'Free WiFi',
 'Free WiFi',
 'Follows Covid-19 sanitation guidance',
 'Free WiFi']

In [523]:
len(facilities)

19

In [527]:
# Creating DataFrame for scrapped data

data_list= list(zip(hostel_name,distance,facilities, url))

data= pd.DataFrame(data_list, columns =['hostel_name','distance','facilities', 'url'],
                              index= pd.RangeIndex(start=1, stop=20, name='Sr. No.'))

#Print the required data

data.to_csv("hotels_details.csv", index= False)

print("Details of all the available Hostels from https://www.hostelworld.com/ in “London” location:\n")
data

Details of all the available Hostels from https://www.hostelworld.com/ in “London” location:



,hostel_name,distance,facilities,url
Sr. No.,,,,
1,Hostel One Notting Hill,5.5km from city centre,Free WiFi,https://www.hostelworld.com/pwa/hosteldetails....
2,St Christopher's Village,1.8km from city centre,"Free WiFi,Follows Covid-19 sanitation guidance",https://www.hostelworld.com/pwa/hosteldetails....
3,Generator London,3km from city centre,"Free WiFi,Follows Covid-19 sanitation guidance",https://www.hostelworld.com/pwa/hosteldetails....
4,Safestay London Kensington Holland Park,5.9km from city centre,Free WiFi,https://www.hostelworld.com/pwa/hosteldetails....
5,St Christopher's Camden,4.3km from city centre,"Free WiFi,Follows Covid-19 sanitation guidance",https://www.hostelworld.com/pwa/hosteldetails....
6,NX London Hostel,6.1km from city centre,"Free WiFi,Free Breakfast",https://www.hostelworld.com/pwa/hosteldetails....
7,St Christopher's Hammersmith,7.5km from city centre,"Free WiFi,Follows Covid-19 sanitation guidance",https://www.hostelworld.com/pwa/hosteldetails....
8,Smart Hyde Park View Hostel,5km from city centre,"Free WiFi,Follows Covid-19 sanitation guidance",https://www.hostelworld.com/pwa/hosteldetails....
9,Saint James Backpackers,5.5km from city centre,"Free WiFi,Free Breakfast,Follows Covid-19 sani...",https://www.hostelworld.com/pwa/hosteldetails....


In [528]:
driver.close()